In [1]:
!git clone https://github.com/Jaish19/GenAI---RAG-using-LangChain.git

Cloning into 'GenAI---RAG-using-LangChain'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 89.09 KiB | 480.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


**STEPS OF RAG PIPELINE PROCESS:**

1. Create the Database and vectorize the document to learn and put the embedding data to DB (FAISS)
2. Create a retriever tool to pull the data from db based on query it gets.
3. Create the ChatOpenAI llm model to put the query + retriever response to llm model to get the response.
4. Create the Langchain framework to organize this entire process to work seemless.


In [2]:
!pip install -U langchain
!pip install -U langchain_community
!pip install -U langchain_openai
!pip install faiss-cpu
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Fetching a document - "Atomic habits"
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader(r"/content/GenAI---RAG-using-LangChain/Atomic_habits.pdf")
pdfpages = pdfloader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
import os




In [4]:
from google.colab import userdata
sec_key=userdata.get("OPENAI_API_KEY")
print(sec_key)

sk-proj-xJPFWrFD0CU1P7PsRNCaT3BlbkFJY2xPYjYFMLJUIVzPjpNi


In [5]:
import os
os.environ["OPENAI_API_KEY"] = sec_key

In [6]:
# load the document using langchain's built in extractors, formatters, loaders, embeddings, and LLMs

mydocuments = pdfloader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(mydocuments)
print(texts)

# using openAI default embeddingModel

embeddings = OpenAIEmbeddings()

# Using Langchain API function to interact with FAISS
db = FAISS.from_documents(texts, embeddings)


[Document(page_content='ATOMIC HABITS  \n"Atomic Habits" is a self -help book written by James Clear. The book offers a framework for \nbuilding good habits and breaking bad ones by focusing on small, incremental changes. The \nmain idea is that small, consistent changes can lead to significant improvements over time. \nClear divides habits into four parts: the cue, the craving, the response, and the reward. He \nalso emphasizes the importance of creating an environment that supports good habits and \nmaking the new habit as easy as possibl e to start. The book also covers the idea of "habit \nstacking" which is the act of using existing habits as a cue to start a new habit. Additionally, \nClear also talks about the idea of "identity -based habits" which is the idea that our habits are \na reflecti on of who we believe we are. Overall, the book provides practical strategies for \ncreating and maintaining habits that stick.', metadata={'source': '/content/GenAI---RAG-using-LangChain/At

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
db

In [ ]:
# Using langchain retriever

retriever = db.as_retriever()
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "My_Fav_Book_is_Atomic_Habbits",
    "Searches and returns documents reg. the book of atomic habbits"
)

tool = [tool]

# wrapping LLM with conversation agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

# Langchain given API to interact with chat models
from langchain_openai.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)
agent_executer = create_conversational_retrieval_agent(llm, tools, verbose=True)


NameError: name 'db' is not defined

In [ ]:
input = "What are the surprising power of atomic habits"
result = agent_executer.invoke({"input":input})